# needed

# database with college football teams, colors, logos, game info (https://collegefootballdata.com/exporter)

# open ai key so I can use their image generation (https://www.geeksforgeeks.org/generate-images-with-openai-in-python/)

# s3 so I can save designs

 


In [1]:

from PIL import Image, ImageOps, ImageDraw, ImageFont
import numpy as np
import warnings
import requests
import pandas as pd
import json
import datetime as dt
import cfbd
import helpers as hf
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import yaml
import random
import base64
import svgwrite
from io import BytesIO

In [2]:
# head = {
# "Authorization": 'Bearer',
# "Content-Type": "application/json"
# }

# url = "https://api.printify.com/v1/shops/13647323/products/65a37b41feca5ad9cb0c1a22.json"

# resp = requests.get(url = url, headers=head)


In [3]:
# json.loads(resp.text)

In [4]:
def process_game(game,teams,design_config,s3_mover,game_id,test,shop_config,pref):
    print(f'    generating game image')
    game = game.to_dict('records')[0]
    game_config = hf.parse_game(game,teams)
    game_config['game_id'] = game
    config = hf.combine_configs(design_config,game_config)
    design,text = hf.build_design(config,test=False)

    # do I need to save all my images to s3?
    s3_mover.image_to_s3(design,rf"{pref}/{game_config['home_team']['shortn']}_v_{game_config['away_team']['shortn']}_{game_id}")

    if not test:
        title,description,tags=hf.generate_t_d_t(game_config)
        shop_config['image'] = design
        shop_config['title'] = title
        shop_config['description'] =  description
        shop_config['tags'] =  tags
        shop_config['design'] = design
        shop_config['text'] = text
        print(shop_config)
        hf.post(shop_config,publish=True)

In [5]:
def monday_run(test=False,specific_game = None):
    design_config,shop_config = hf.get_config()
    s3_mover = hf.s3_mover()
    cfbd_loader = hf.cfbp_handler(design_config['cfbd_api'])
    yr = dt.date.today().year
    yr=2023

    # refresh teams if new season, could also add something to check if any teams exist
    if hf.new_football_season():
        teams = cfbd_loader.get_team_info()
        s3_mover.pd_to_s3(teams,f'teams/all_teams_{yr}')
    else:
        # get my df of teams
        teams = s3_mover.s3_to_pd(f'teams/all_teams_{yr}')
    
    # get new schedule info and return list of games i need to do
    all_games = cfbd_loader.get_schedule(yr)
    old_games = s3_mover.s3_to_pd(s3_name=f'old/all_games_{yr}')
    to_do_new,to_do_upcoming = cfbd_loader.determine_to_do(old_games)

    if specific_game is not None:
        to_do_new = []
        to_do_upcoming = [specific_game]

    print(f'New games to do {len(to_do_new)}')
    print(f'Upcoming games to do {len(to_do_upcoming)}')

    # have to line image path for config up with
    print('new games')
    for game_id in to_do_new:
        game = all_games.loc[all_games.id == game_id]
        process_game(game,teams,design_config,s3_mover,game_id,test,shop_config,pref='post')

    print('upcoming games')
    for game_id in to_do_upcoming:
        game = all_games.loc[all_games.id == game_id]
        process_game(game,teams,design_config,s3_mover,game_id,test,shop_config,pref='pre')

    s3_mover.pd_to_s3(all_games,s3_name=f'old/all_games_{yr}')

In [7]:
monday_run(test=False,specific_game=401551788)

New games to do 0
Upcoming games to do 1
new games
upcoming games
    generating game image
NCAA football University Of Washington Huskies mascot beating University Of Texas Longhorns mascot
using saved image because test mode
CFB_shirt/images/pre/Washington_v_Texas_401551788.png
{'base_url': 'https://api.printify.com/v1', 'shop_id': 13647323, 'title': 'Washington VS Texas. 02 Jan, 2024', 'price': 2000, 'description': 'AI image representing the football game 37 to 31 between Washington and Texas, to be played 02 Jan, 2024', 'tags': 'football,college,sports,Washington,Texas', 'blueprint_id': 6, 'print_provider_id': 87, 'variant_ids': [12070, 12071, 12072, 12073, 12074, 12100, 12101, 12102, 12103, 12104], 'shop_name': '70f8b8-5', 'printify_access': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJhdWQiOiIzN2Q0YmQzMDM1ZmUxMWU5YTgwM2FiN2VlYjNjY2M5NyIsImp0aSI6ImU2YzkwZTk2NjZiMDc0NjI4NjliNjAzMDM1MGMwOWQxZTkxMjgyMTgxNzVmNmI1MWIzNmQ2OWM5YjExNDYwY2U1NjdlOTU1ZjdiYjZjNWQyIiwiaWF0IjoxNzA0Njc2NTU0LjkxMTE1N

- handle nan in teams table
- font auto sizing

In [ ]:
#automate adding shirts to printfy/shopify

In [ ]:
def get_items(created, post_dict):
    '''
    get items, so we can filter by name to get shopify id to add to collection
    created is a list of all the things we just made to add to the list
    '''
    shopify_headers ={
    "X-Shopify-Access-Token": post_dict["shopify_access"],
    "Content-Type": "application/json",
    }
    products_link = f"https://{post_dict['shop_name']}.myshopify.com/admin/api/2024-01/products.json"

    response1 = requests.get(products_link, headers=shopify_headers)
    print('got all items')
    id_list=[]
    for product in response1.json()['products']:
        if product['title'] in created:
            id_list.append(product['id'])
    print('filtered list for just this collection')
    return id_list


def create_collection(created,week,year,post_dict):
    '''
    give this a list of product names (created) and it will add them to a collection for the week

    add something for the whole has the game happened yet aspect
    '''
    shopify_headers ={
    "X-Shopify-Access-Token": post_dict["shopify_access"],
    "Content-Type": "application/json",
    }
    collection_link = f"https://{post_dict['shop_name']}.myshopify.com/admin/api/2023-04/custom_collections.json"

    id_list = get_items(created, post_dict)
    collects = [{"product_id":x} for x in id_list]
    collection_data = {
    "custom_collection":{
        "title":f"Week {week}, {year}",
        "collects":collects
        }
    }
    response1 = requests.post(collection_link, headers=shopify_headers, json = collection_data)
    if response1.status_code==201:
        print("collection created")
    else:
        print("could not create collection")
        print(json.loads(response1.text))

In [ ]:
# create_collection([shop_config['title']],1,2023,shop_config)